## Datebase query example
This notebooks shows the dafault way to access object stored in your Home Assistant database.

### Setup

In [22]:
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker

from homeassistant.components.recorder.models import Base, Events, States, RecorderRuns

In [23]:
# This requires Home Assistant 0.24+
# Your database URL as specified in configuration.yaml
# If using default settings, it's sqlite:///<path to config dir>/home-assistant_v2.db
DB_URL = "sqlite:////path/to/.homeassistant/home-assistant_v2.db"

In [24]:
# Database setup
engine = create_engine(DB_URL)
Base.metadata.create_all(engine)
session_factory = sessionmaker(bind=engine)
Session = scoped_session(session_factory)

### Available entities and their total number of unique states

In [25]:
list(engine.execute("SELECT entity_id, COUNT(*) FROM states GROUP BY entity_id"))

[('binary_sensor.bathroom_door', 179),
 ('binary_sensor.movement', 38),
 ('gpsd.gps', 771),
 ('gpsd.gpsd', 1),
 ('group.all_lights', 226),
 ('group.all_locks', 537),
 ('group.all_switches', 1689),
 ('group.ha_host', 179),
 ('group.info', 179),
 ('group.kitchen', 179),
 ('group.livingroom', 179),
 ('group.livingroom1', 179),
 ('group.office', 179),
 ('group.time', 179),
 ('input_boolean.control_switch', 41),
 ('input_boolean.notify_home', 182),
 ('input_slider.brightness', 187),
 ('input_slider.darkness', 179),
 ('light.1afe34d535e4', 69),
 ('lock.front_gate', 179),
 ('sensor.bathroom111_temperature', 2),
 ('sensor.battery_tablet', 210),
 ('sensor.control_input', 49),
 ('sensor.cpu', 7705),
 ('sensor.cpu_temp', 2119),
 ('sensor.danotage', 2),
 ('sensor.date', 186),
 ('sensor.discharge', 50),
 ('sensor.glances11', 100),
 ('sensor.ha_release', 130),
 ('sensor.internet_time', 3600),
 ('sensor.kitchen_humidity', 179),
 ('sensor.kitchen_temperature', 205),
 ('sensor.last', 112),
 ('sensor.la

### Get available runs

In [26]:
[str(r.start) for r in Session().query(RecorderRuns)]

['2016-07-15 09:12:19.592055',
 '2016-07-15 09:26:00.942845',
 '2016-07-15 09:32:06.724139',
 '2016-07-15 09:33:25.235434',
 '2016-07-15 09:47:25.311636',
 '2016-07-15 10:01:51.460298',
 '2016-07-15 10:05:25.841796',
 '2016-07-15 10:08:06.893725',
 '2016-07-15 14:06:26.388940',
 '2016-07-15 14:31:19.874527',
 '2016-07-15 14:34:38.110095',
 '2016-07-15 14:36:04.907501',
 '2016-07-15 14:37:19.463373',
 '2016-07-15 14:39:47.683495',
 '2016-07-15 14:40:42.347384',
 '2016-07-15 14:49:06.979299',
 '2016-07-15 14:53:05.294371',
 '2016-07-15 14:57:24.514303',
 '2016-07-15 14:59:54.028440',
 '2016-07-15 15:02:06.632085',
 '2016-07-15 15:03:01.883560',
 '2016-07-15 15:05:46.992554',
 '2016-07-15 16:03:40.601593',
 '2016-07-16 07:48:47.391476',
 '2016-07-16 08:17:47.066920',
 '2016-07-16 08:36:29.846933',
 '2016-07-16 08:38:47.758929',
 '2016-07-16 08:41:24.527377',
 '2016-07-16 08:43:59.797913',
 '2016-07-16 08:45:19.766744',
 '2016-07-16 08:46:04.050954',
 '2016-07-16 08:49:14.389891',
 '2016-0

In [27]:
# Last finished run
last_run = (Session().query(RecorderRuns)
            .filter(RecorderRuns.end != None)
            .order_by(RecorderRuns.end.desc())
            .first().to_native())

### Show states at point in time

In [28]:
from datetime import timedelta, datetime
from sqlalchemy import and_, func

point_in_time = datetime(2016, 7, 4, 9, 0, 0)

# Taken from homeassistant.components.history#get_states
most_recent_state_ids = (
    Session().query(func.max(States.state_id).label('max_state_id'))
    .filter(States.created < point_in_time).group_by(States.entity_id).subquery()
)

query = Session().query(States).join(
    most_recent_state_ids, and_(
        States.state_id == most_recent_state_ids.c.max_state_id))

for state in query:
    # Convert to HA object
    state = state.to_native()
    print(state.name, ":", state.state, state.attributes.get('unit_of_measurement', ''))